In [1]:
# External libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
import astropy
import scipy
from filterpy.kalman import KalmanFilter 
from filterpy.common import Q_discrete_white_noise
from scipy.linalg import block_diag
from astropy import units as u
from poliastro.bodies import Earth, Mars, Sun, Moon
from poliastro.twobody import Orbit
from poliastro.plotting import OrbitPlotter2D
from poliastro.plotting import OrbitPlotter3D
import glob
# Own Libraries
from utility.utils import *
from KalmanFilter.kf import *
from Detect.detector import *
from Match.pair import *
from Match.icp import *

%matplotlib tk
style.use('seaborn-paper')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [2]:
############################################
dt = 1
############################################
df = pd.read_csv(f'DATA/ephemeris sat/inclination zero/{dt} step size.csv', header=3, sep=';') 
real_Latitudes, real_Longitudes, real_Altitudes = df['Lat (deg)'], df['Lon (deg)'], df['Alt (km)']
real_Vxs,real_Vys,real_Vzs = df['x (km/sec)'], df['y (km/sec)'],df['z (km/sec)']

real_X, real_Y, real_Z = [], [], []
for i in range(len(df)):
    altitude = real_Altitudes[i]
    latitude = real_Latitudes[i]
    longitude = real_Longitudes[i]
    x, y, z = spherical2cartesian(altitude, latitude, longitude)
    real_X.append(x)
    real_Y.append(y)
    real_Z.append(z)
real_X, real_Y, real_Z = np.array(real_X),np.array(real_Y),np.array(real_Z)

In [3]:
# plt.figure(dpi=150, tight_layout=True)

# sub_x = plt.subplot(3,1,1)
# plt.title('Satellite True Position')
# plt.plot(real_X, 'k', linewidth=0.8)
# plt.ylabel('X')

# sub_y = plt.subplot(3,1,2, sharex = sub_x)
# plt.plot(real_Y,'k',linewidth=0.8)
# plt.ylabel('Y')

# sub_z = plt.subplot(3,1,3, sharex = sub_x)
# plt.plot(real_Z,'k',linewidth=0.8)
# plt.xlabel('Time step (0.1 sec)')
# plt.ylabel('Z')
# plt.ylim([-1,1])

# plt.show()

In [4]:
# Initial State Vector:
x, y, z = real_X[0], real_Y[0], real_Z[0]
vx, vy, vz = real_Vxs[0], real_Vys[0], real_Vzs[0]

In [5]:
kf = KalmanFilter(dim_x = 6, dim_z = 3)
# Offset State Vector:
iX, iY, iZ, iVx, iVy, iVz  = 0,1,2,3,4,5
kf.F[iX, iVx] = dt
kf.F[iY, iVy] = dt
kf.F[iZ, iVz] = dt
q = Q_discrete_white_noise(dim=3, dt=dt, var=0.001)
kf.Q = block_diag(q, q)
kf.H[iX, iX] = 1
kf.H[iZ, iZ] = 1
kf.H[iY, iY] = 1
kf.R *= 5 # KM 
kf.x = np.array([[x, y, z, vx, vy, vz]]).T
kf.u = 0
kf.P = np.eye(6) * 500

In [6]:
# LOAD ALL IMAGES:
dict = {}
for img in glob.glob(f"DATA/ephemeris sat/inclination zero/{dt} step size/*"):
    txt = img             # stringa
    t = txt.split('_')[1] # numero
    dict[t] = txt

In [7]:
mus = []
Zs = []
STEPS = len(df)
for idx in range(STEPS):
    if idx == 0:
        # UPDATE
        img=cv2.imread(dict[str(idx+1)])
        craters_det = detect(img) # x,y, radius(px)
        craters_cat = crater_catalogued([x,y,z]) # OUTPUTS: x,y, radius (px)
        FEATURE, indexes = crater_match([x,y,z], craters_det) # Feature Matched in LCLF frame
        Z = position_estimation([x,y,z], craters_det, craters_cat, indexes)
        kf.update(Z)
        Zs.append(Z)


    else:    
        kf.predict()
        x,y,z = float(kf.x[0]), float(kf.x[1]), float(kf.x[2])
        # UPDATE
        img=cv2.imread(dict[str(idx+1)])
        craters_det = detect(img) # x,y, radius(px)
        craters_cat = crater_catalogued([x,y,z]) # OUTPUTS: x,y, radius (px)
        # FEATURE, indexes = crater_match([x,y,z], craters_det) # Feature Matched in LCLF frame
        indexes = matching(craters_det, craters_cat)
        if indexes is not None:
            Z = position_estimation([x,y,z], craters_det, craters_cat, indexes)
            kf.update(Z)
            Zs.append(Z)
        
        else:
            Zs.append(np.array([0,0,0]))

    mus.append(kf.x)

No crater found


AttributeError: 'NoneType' object has no attribute 'shape'

In [8]:
x_pred = []
for mu in mus:
    x = mu[0]
    x_pred.append(x)
x_true = real_X[:len(x_pred)]

y_pred = []
for mu in mus:
    y = mu[1]
    y_pred.append(y)
y_true = real_Y[:len(y_pred)]

z_pred = []
for mu in mus:
    z = mu[2]
    z_pred.append(z)
z_true = real_Z[:len(z_pred)]
####################################################################
######################## - MEAS - ##################################
X_m = []
for i in range(len(Zs)):
    X_m.append(Zs[i][0])

Y_m = []
for i in range(len(Zs)):
    Y_m.append(Zs[i][1])

Z_m = []
for i in range(len(Zs)):
    Z_m.append(Zs[i][2])

In [9]:
lw = 0.8

plt.figure(dpi=150)
plt.subplot(311)
plt.plot(x_pred, '-k', linewidth=lw)
plt.plot(x_true, 'r', linewidth=lw)
plt.scatter(range(len(X_m)),X_m)
plt.legend(['pred','true','meas'])
plt.xlabel(f'Step Size: {dt}')
plt.ylim([-1520,-1200])
plt.ylabel('X')
plt.show()

plt.subplot(312)
plt.plot(y_pred, '-k', linewidth=lw)
plt.plot(y_true, 'r', linewidth=lw)
plt.scatter(range(len(X_m)),Y_m)
plt.legend(['pred','true','meas'])
plt.xlabel(f'Step Size: {dt}')
plt.ylabel('Y')
plt.show()

plt.subplot(313)
plt.plot(z_pred, '-k', linewidth=lw)
plt.plot(z_true, 'r', linewidth=lw)
plt.scatter(range(len(Z_m)),Z_m)
plt.legend(['pred','true','meas'])
plt.xlabel(f'Step Size: {dt}')
plt.ylabel('Z')
plt.show()

In [10]:
#plt.figure(dpi=150)
#plt.plot(x_pred, '-k')
#plt.scatter([int(i*MES) for i in range(int(N/MES))], [Xs[i*MES] for i in range(int(N/MES))])
#plt.plot(x_true, 'r')
#plt.legend(['pred','true','meas'])
#plt.xlim([1000-1,1000+1])
#plt.ylim([100-5,100+5])
#plt.xlabel('Time')
#plt.ylabel('X')
#plt.show()

In [11]:
plt.figure(dpi=150, tight_layout=True, figsize=(5,10))

plt.subplot(311)
x_pred = np.array(x_pred)
x_true = np.array(x_true)
diff = []
for x,y in zip(x_pred,x_true):
    d = abs(x - y)
    diff.append(d)
plt.title('Position - x')
plt.plot(diff, '-k')
plt.ylabel('Error Km')
plt.show()

plt.subplot(312)
y_pred = np.array(y_pred)
y_true = np.array(y_true)
diff = []
for x,y in zip(y_pred,y_true):
    d = abs(x - y)
    diff.append(d)
plt.title('Position - y')
plt.plot(diff, '-k')
plt.ylabel('Error Km')
plt.show()

plt.subplot(313)
z_pred = np.array(z_pred)
z_true = np.array(z_true)
diff = []
for x,y in zip(z_pred,z_true):
    d = abs(x - y)
    diff.append(d)
plt.title('Position - z')
plt.plot(diff, '-k')
plt.xlabel(f'Step Size: {dt}')
plt.ylabel('Error Km')
plt.show()